In [ ]:
#@title Supervised Learning { display-mode: "form" }
#@markdown This praktikum re-analyses gene expression measurements of two leukemia subtypes: acute lymphoblastic leukemia (ALL) and acute myeloid leukemia (AML) from ([Golub et al. 1999](https://doi.org/10.1126/science.286.5439.531)).
#@markdown AML and ALL are treated with different drugs (May 9th quiz, last question).
#@markdown Determining the leukemia subtype involves multiple tests, and an experienced hematopathologist to interpet them.
#@markdown Here we aim to use supervised learning to train a classifier and predict the leukemia subtype from a single gene expression experiment.

#@markdown The web site with the raw data referred to in ([Golub et al. 1999](https://doi.org/10.1126/science.286.5439.531)) is no longer accessible.
#@markdown Luckily, the data can be found from other sources, we will use a version included in the [CAMAN package](https://cran.r-project.org/web/packages/CAMAN/index.html).
#@markdown The current cell will set up the environment & load the gene expression measurements (`exprs`), and corresponding true labels as determined by clinical tests (`sample_labels`).

#@markdown Some code cells will contain a red question mark (❓).
#@markdown To make the most of the praktikum, try to fill in the missing code.
#@markdown Typically, this will involve copying a few lines directly from above, and replacing variables or adjusting parameters.

# Remove Colab `/content/sample_data/` directory to avoid confusion
!rm -rf sample_data

# Upgrade scikit-learn, as the default Colab version does not have `DecisionBoundaryDisplay`
!pip install pyreadr

# Download & unpack the CAMAN package
!wget -nv https://cran.r-project.org/src/contrib/CAMAN_0.77.tar.gz
!tar xvfz CAMAN_0.77.tar.gz > /dev/null

# Import modules from numpy, pandas, seaborn, sklearn
import requests
import numpy as np, pandas as pd, seaborn as sns
import matplotlib.pyplot as plt
import sklearn.inspection, sklearn.metrics, sklearn.neighbors, sklearn.svm, sklearn.pipeline, sklearn.preprocessing
import pyreadr

# Fix `RANDOM_SEED` for (partial) reproducibility
RANDOM_SEED = 4 # https://xkcd.com/221

def plot_question_mark(ax):
  ax.text(x=.5, y=.5, s='?', color='red', fontsize=64,
          horizontalalignment='center',
          verticalalignment='center',
          transform=ax.transAxes)

# Use g:Convert to map microarray probe identifiers to gene names
def g_convert(query, target='ENSG', organism='hsapiens', simplify=False):
  # https://biit.cs.ut.ee/gprofiler/convert
  r = requests.post(url='https://biit.cs.ut.ee/gprofiler/api/convert/convert/', json=locals())
  df = pd.DataFrame(r.json()['result'])
  return df

golubMerge = pyreadr.read_r('CAMAN/data/golubMerge.RData')
exprs = golubMerge['golubMerge.exprs']
sample_labels = golubMerge['sample.labels']
print('Loading Golub et al. 1999 data:')
print('- exprs:', exprs.shape)
print('- sample_labels:', sample_labels.shape)

# 1 Data wrangling

## Expression measurements
The gene expression matrix `exprs` has 7129 rows and 72 columns. The number of columns matches the number of samples in the training set (38) plus the number of samples in the validation set (34).

In [ ]:
exprs.head()

The row names look like Affymetrix probe identifiers. We will use [g:Convert](https://biit.cs.ut.ee/gprofiler/convert) to transform them to gene names. We will also aggregate the probe-level measurements by taking the average of probes mapping to the same gene. 

In [ ]:
gene_conversion = g_convert(exprs.index.tolist()).query('(n_converted == 1) & (name != "None")')
gene_expression = exprs.merge(gene_conversion[['incoming', 'name']], left_index=True, right_on='incoming').drop(['incoming'], axis=1).set_index('name', drop=True).groupby('name').agg('mean')
gene_expression

How many gene-level measurements do we get? How does the number compare to what was reported in the original study (n=6817)? If the numbers are different, why?

## Sample labels
According to the paper, the training set has 38 samples (27 ALL, 11 AML) and the validation set has 34 samples (20 ALL, 14 ALL; footnote 23). The total number of ALL and AML samples from the paper seems to match the contents of `sample_labels`.

In [ ]:
sample_labels.value_counts()

However, there's no explicit information on whether a sample belongs to the training set or the validation set. We first try using the first 38 samples as the training set (`samples_train`), and the remaining 34 samples as the validation set (`samples_valid`). This does not reproduce the number of ALL/AML samples in the training/validation sets as reported in the paper. Can you find an alternative partitioning that recapitulates ALL/AML counts? Look at the contents of `sample_labels` for additional clues.

We then define training/validation subsets of gene expression and corresponding labels to use as input for supervised learning.

In [ ]:
#❓: Modify lines below to reproduce the train/validation sets from the original study
# first 38 samples as the training set (`samples_train`), and the remaining 34 samples as the validation set (`samples_valid`)
samples_train = range(0, 38)
samples_valid = range(38, len(sample_labels))

# Sample labels for training & validation sets (check counts)
labels_train = sample_labels.loc[samples_train]
labels_valid = sample_labels.loc[samples_valid]
print('Labels, training:', labels_train.value_counts())
print('Labels, validation:', labels_valid.value_counts())

# Gene expression for training & validation sets
expression_train = gene_expression.transpose().loc[samples_train]
expression_valid = gene_expression.transpose().loc[samples_valid]
print('Gene expression, training:', expression_train.shape)
print('Gene expression, validation:', expression_valid.shape)

# Gene expression + sample label together for traning & validation sets
# (Useful for seabron visualisation routines.)
data_train = pd.concat([expression_train, labels_train], axis=1)
data_valid = pd.concat([expression_valid, labels_valid], axis=1)

# 2 Single-feature classification

The original study notes that ALL/AML subtypes were initially motivated by differences in periodic acid-Schiff (PAS) staining, and levels of myeloperoxidase. Could we differentiate between ALL/AML by using known biologically relevant genes?
PAS staining in cancer cells detects excess glycogen, we somwehat arbitrarily pick HK2 as a PAS proxy. Similarly, we also look at expression levels for myeloperoxidase (MPO).

In [ ]:
gene1, gene2 = 'HK2', 'MPO'

# Plot with two sub-panels
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(6, 3), constrained_layout=True)

# Violin plot for GCK glucokinase
ax1.set_title(gene1)
sns.violinplot(data=data_train, x=gene1, y='sample.labels', ax=ax1);

#❓: Add violin plot for MPO myeloperoxidase
plot_question_mark(ax2)

Gene expression looks different for both subgroups, let's look at the corresponding ROC curves and AUC scores.

In [ ]:
# Plot with two sub-panels
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(6, 3), constrained_layout=True)

# Add ROC curve for GCK
ax1.set_title(gene1)
ax1.set_aspect('equal')
sklearn.metrics.RocCurveDisplay.from_predictions(y_true=labels_train, y_pred=expression_train[gene1], pos_label='ALL', ax=ax1);

#❓: Add ROC curve for MPO
plot_question_mark(ax2)

# 3 Visualising classifier output
Both HK2 and MPO expression are different in ALL/AML, but neither alone would be sufficient for a clinically useful classification. We'll try to combine the two genes using a supervised learning method. Specifically, we will try a linear Support Vector Machine (SVM) and k-nearest neighbours classifier (KNN), as discussed in [this article](https://www.nature.com/articles/nmeth.4551) from the [Points of Significance series](https://www.nature.com/collections/qghhqm/pointsofsignificance).

First, we train an SVM on HK2 and MPO expression, and visualise the corresponding decision surface and ROC curve for the training data.

In [ ]:
# Train SVM on training data
svm = sklearn.svm.SVC(kernel='linear', random_state=RANDOM_SEED)
svm.fit(X=expression_train[[gene1, gene2]], y=np.ravel(labels_train))

# Plot with two sub-panels
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(6, 3), constrained_layout=True)

# Left: decision boundary with training data
sklearn.inspection.DecisionBoundaryDisplay.from_estimator(estimator=svm, X=expression_train[[gene1, gene2]], alpha=0.5, ax=ax1);
sns.scatterplot(data=data_train, x=gene1, y=gene2, hue='sample.labels', ax=ax1);

# Right: ROC curve (training data)
sklearn.metrics.RocCurveDisplay.from_estimator(svm, X=expression_train[[gene1, gene2]], y=labels_train, pos_label='AML', ax=ax2);

We'll also train a KNN classifier for comparison. We have changed n_neighbors from the default value (5) to 2, leading to a very high AUC score on the training data.

In [ ]:
# Train knn on training data
knn = sklearn.neighbors.KNeighborsClassifier(n_neighbors=2)
knn.fit(X=expression_train[[gene1, gene2]], y=np.ravel(labels_train))

# Plot with two sub-panels
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(6, 3), constrained_layout=True)

#❓: Add decision boundary for knn 
plot_question_mark(ax1)

#❓: Add ROC curve for knn
plot_question_mark(ax2)

# 4 Training vs validation

We'll now look at the performance of both models on the validation data. Which classifier would you expect to generalise better (e.g. looking at the decision boundaries above)?

In [ ]:
# Plot with two sub-panels
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(6, 3), constrained_layout=True)

# Left: decision boundary with test data
sklearn.inspection.DecisionBoundaryDisplay.from_estimator(estimator=svm, X=expression_valid[[gene1, gene2]], alpha=0.5, ax=ax1);
sns.scatterplot(data=data_valid, x=gene1, y=gene2, hue='sample.labels', ax=ax1);

# Right: ROC curve (test data)
sklearn.metrics.RocCurveDisplay.from_estimator(svm, X=expression_valid[[gene1, gene2]], y=labels_valid, pos_label='AML', ax=ax2, name='linear SVM');

How would you expect KNN to perform when changing n_neighbors back to the default value (and re-training)?

In [ ]:
# Plot with two sub-panels
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(6, 3), constrained_layout=True)

#❓: Add decision boundary for KNN on validation data
plot_question_mark(ax1)

#❓: Add ROC curve for KNN on validation data
plot_question_mark(ax2)

# 5 High-dimensional data

We now switch to working with expression data from all genes. Running an SVM with default settings reproduces the 100% accuracy reported in the study. How does this change when using a different value for the regularization parameter C?

In [ ]:
# Plot with two sub-panels
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(8, 4), constrained_layout=True)
ax1.set_aspect('equal')
ax2.set_aspect('equal')

svm_full = sklearn.svm.SVC(kernel='linear', random_state=RANDOM_SEED)
svm_full.fit(X=expression_train, y=np.ravel(labels_train))

sklearn.metrics.RocCurveDisplay.from_estimator(svm_full, expression_train, labels_train, ax=ax1, name='SVM training');
sklearn.metrics.RocCurveDisplay.from_estimator(svm_full, expression_valid, labels_valid, ax=ax1, name='SVM validation');

sklearn.metrics.ConfusionMatrixDisplay.from_estimator(estimator=svm_full, X=expression_valid, y=labels_valid, ax=ax2);

How does a KNN classifier perform when trained on all genes? Feel free to try out different values for n_neighbors, like for the regularization parameter C for the SVM above. Can you rationalise the differences between an SVM and a KNN when working with a large number of features (mentioned in [this article](https://www.nature.com/articles/nmeth.4551))?

In [ ]:
# Plot with two sub-panels
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(8, 4), constrained_layout=True)
ax1.set_aspect('equal')
ax2.set_aspect('equal')

#❓: Train a KNN on the full data set
# knn_full = sklearn.neighbors.KNeighborsClassifier(...
# knn_full.fit(...

#❓: Plot ROC on train & validation data
plot_question_mark(ax1)

#❓: Plot confusion matrix on validation data
plot_question_mark(ax2)

# 6 Model interpretability

Many machine learning algorithms are able to estimate the contribution of each feature towards making correct classifications. In the original study, feature importance was quantified by correlating the expression of an individual gene to the sample labels. For a linear SVM, feature importance is often quantified with the coefficients of the separating hyperplane. The code below extracts the hyperplane weights (one per gene), normalises and ranks them.

In [ ]:
feature_importance = expression_train.transpose()
feature_importance.insert(loc=0, column='SVM_importance', value=svm_full.coef_[0] / max(abs(svm_full.coef_[0])))
feature_importance.insert(loc=1, column='SVM_importance_pct', value=feature_importance['SVM_importance'].abs().rank(ascending=True, pct=True))
feature_importance = feature_importance.sort_values('SVM_importance_pct', ascending=False)
feature_importance.head(10)

The two tables below show the top 5 ALL-enriched, and AML-enriched genes from Figure 3B of the original study (using updated gene names). Can you discuss the agreement between 

Conversely, can you find any additional evidence for the top SVM-supported genes in ALL or AML, either from the original study or from additional literature?

In [ ]:
all_genes = [
  'MYB', # C-myb
  'PSMA6', # Proteasome iota
  'CD79A', # MB-1
  'CCND3', # Cyclin D3
  'MYL6B', # Myosin light chain
]
feature_importance.query('name in @all_genes')

In [ ]:
aml_genes = [
  'FAH', # Fumarylacetoacetate
  'ZYX', # Zyxin
  'LTC4S', # LTC4 synthase
  'LYN', # LYN
  'HOXA9', #HoxA9
]
#❓: Show SVM weights for top AML genes